In [1]:
%matplotlib inline
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
n_iter = 0
for i in xrange(112):
    for j in xrange(i+1,111):
        n_iter += 1
print n_iter, i, j

6105 111 110


# Градиентынй бустинг "в лоб"

В этой части работы решение получается с помощью градиентного бустинга.

In [3]:
features = pd.read_csv('dota_train.csv')#считывание файла features
features_test = pd.read_csv('dota_test.csv')#считывание test_features

In [4]:
features.head()#топ строк для просмотра структуры

,match_id,start_time,lobby_type,r1_hero,r1_level_0,r1_xp_0,r1_gold_0,r1_lh_0,r1_kills_0,r1_deaths_0,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
0,0,1430198770,7,11,0,0,0,0,0,0,...,4,2,2,-52,2874,1,1796,0,51,0
1,1,1430220345,0,42,1,0,0,0,0,0,...,4,3,1,-5,2463,1,1974,0,63,1
2,2,1430227081,7,33,0,0,0,0,0,0,...,4,3,1,13,2130,0,0,1830,0,63
3,3,1430263531,1,29,0,0,0,0,0,0,...,4,2,0,27,1459,0,1920,2047,50,63
4,4,1430282290,7,13,1,0,0,0,0,0,...,3,3,0,-16,2449,0,4,1974,3,63


In [5]:
features_test.head()#топ строк для просмотра структуры

,match_id,start_time,lobby_type,r1_hero,r1_level_0,r1_xp_0,r1_gold_0,r1_lh_0,r1_kills_0,r1_deaths_0,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,6,1430287923,0,93,0,0,0,0,0,0,...,0,12,247,-86,272,3,4,2,0,118
1,7,1430293357,1,20,1,0,0,0,0,0,...,2,-29,168,-54,NaN,3,2,2,1,16
2,10,1430301774,1,112,0,25,50,1,0,0,...,1,-22,46,-87,186,1,3,3,0,-34
3,13,1430323933,1,27,0,0,0,0,0,0,...,2,-49,30,-89,210,3,4,2,1,-26
4,16,1430331112,1,39,0,0,0,0,0,0,...,0,36,180,-86,180,1,3,2,1,-33


In [6]:
count = features.count() #подсчет числа заполненных полей

In [7]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97230 entries, 0 to 97229
Columns: 459 entries, match_id to barracks_status_dire
dtypes: float64(12), int64(447)
memory usage: 341.2 MB


In [8]:
print count #выведем на экран

match_id                       97230
start_time                     97230
lobby_type                     97230
r1_hero                        97230
r1_level_0                     97230
r1_xp_0                        97230
r1_gold_0                      97230
r1_lh_0                        97230
r1_kills_0                     97230
r1_deaths_0                    97230
r1_items_0                     97230
r1_level_60                    97230
r1_xp_60                       97230
r1_gold_60                     97230
r1_lh_60                       97230
r1_kills_60                    97230
r1_deaths_60                   97230
r1_items_60                    97230
r1_level_120                   97230
r1_xp_120                      97230
r1_gold_120                    97230
r1_lh_120                      97230
r1_kills_120                   97230
r1_deaths_120                  97230
r1_items_120                   97230
r1_level_180                   97230
r1_xp_180                      97230
r

# Целевая переменная в radiant win.

Признаки с пропусками:

first_blood_time               77677

first_blood_team               77677

first_blood_player1            77677

first_blood_player2            53243

radiant_bottle_time            81539

radiant_courier_time           96538

radiant_flying_courier_time    69751

radiant_first_ward_time        95394

dire_bottle_time               81087

dire_courier_time              96554

dire_flying_courier_time       71132

dire_first_ward_time           95404

Курьера можно и не покупать (но лучше его иметь, конечно),а уж летающего вообще не обязательно иметь.Поэтому столбцы, связанные с курьером имеют пропуски.

In [9]:
features = features.fillna(value=0)#заполнение пустых полей
features_test = features_test.fillna(value=0)#заполнение пустых полей

In [10]:
from sklearn import cross_validation#импорт кросс-валидации
kf = cross_validation.KFold(len(features),n_folds=5,shuffle=True,random_state=241)#задаем кросс-валидацию

In [11]:
features_target = features['radiant_win']#выбираем целевой столбец

In [12]:
features_train = features.drop(['duration','radiant_win','tower_status_radiant','tower_status_dire','barracks_status_dire','barracks_status_radiant'],axis=1)#удаляем столбцы, которые заглядывают в будущее

In [13]:
features_train.head()#новый датасет без столбцов, которые заглядывают в будущее

,match_id,start_time,lobby_type,r1_hero,r1_level_0,r1_xp_0,r1_gold_0,r1_lh_0,r1_kills_0,r1_deaths_0,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,0,1430198770,7,11,0,0,0,0,0,0,...,0,35,103,-84,221,3,4,2,2,-52
1,1,1430220345,0,42,1,0,0,0,0,0,...,0,-20,149,-84,195,5,4,3,1,-5
2,2,1430227081,7,33,0,0,0,0,0,0,...,1,-39,45,-77,221,3,4,3,1,13
3,3,1430263531,1,29,0,0,0,0,0,0,...,0,-30,124,-80,184,0,4,2,0,27
4,4,1430282290,7,13,1,0,0,0,0,0,...,0,46,182,-80,225,6,3,3,0,-16


In [14]:
features_train.info()
del(features)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97230 entries, 0 to 97229
Columns: 453 entries, match_id to dire_first_ward_time
dtypes: float64(12), int64(441)
memory usage: 336.8 MB


In [15]:
#полезные фичи, дают 0.75540
for i in [0,60,120,180,240,300]:
    features_train['r_level_mean_'+str(i)] = features_train[['r1_level_'+str(i),'r2_level_'+str(i),'r3_level_'+str(i),'r4_level_'+str(i),'r5_level_'+str(i)]].mean(axis=1)
    features_train['d_level_mean_'+str(i)] = features_train[['d1_level_'+str(i),'d2_level_'+str(i),'d3_level_'+str(i),'d4_level_'+str(i),'d5_level_'+str(i)]].mean(axis=1)
    features_train['r_xp_mean_'+str(i)] = features_train[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].mean(axis=1) 
    features_train['d_xp_mean_'+str(i)] = features_train[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].mean(axis=1)
    features_train['r_level_sum_'+str(i)] = features_train[['r1_level_'+str(i),'r2_level_'+str(i),'r3_level_'+str(i),'r4_level_'+str(i),'r5_level_'+str(i)]].sum(axis=1)
    features_train['d_level_sum_'+str(i)] = features_train[['d1_level_'+str(i),'d2_level_'+str(i),'d3_level_'+str(i),'d4_level_'+str(i),'d5_level_'+str(i)]].sum(axis=1)
    features_train['r_xp_sum_'+str(i)] = features_train[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].sum(axis=1) 
    features_train['d_xp_sum_'+str(i)] = features_train[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].sum(axis=1) 
    features_train['r_gold_mean_'+str(i)] = features_train[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].mean(axis=1)
    features_train['d_gold_mean_'+str(i)] = features_train[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].mean(axis=1) 
    features_train['r_gold_sum_'+str(i)] = features_train[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].sum(axis=1)
    features_train['d_gold_sum_'+str(i)] = features_train[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].sum(axis=1) 
    features_train['kills_diff_'+str(i)] = features_train[['r1_kills_'+str(i),'r2_kills_'+str(i),'r3_kills_'+str(i),'r4_kills_'+str(i),'r5_kills_'+str(i)]].sum(axis=1)-features_train[['d1_kills_'+str(i),'d2_kills_'+str(i),'d3_kills_'+str(i),'d4_kills_'+str(i),'d5_kills_'+str(i)]].sum(axis=1)
    features_train['r_lh_mean_'+str(i)] = features_train[['r1_lh_'+str(i),'r2_lh_'+str(i),'r3_lh_'+str(i),'r4_lh_'+str(i),'r5_lh_'+str(i)]].mean(axis=1) 
    features_train['d_lh_mean_'+str(i)] = features_train[['d1_lh_'+str(i),'d2_lh_'+str(i),'d3_lh_'+str(i),'d4_lh_'+str(i),'d5_lh_'+str(i)]].mean(axis=1) 
    features_train['r_lh_sum_'+str(i)] = features_train[['r1_lh_'+str(i),'r2_lh_'+str(i),'r3_lh_'+str(i),'r4_lh_'+str(i),'r5_lh_'+str(i)]].sum(axis=1) 
    features_train['d_lh_sum_'+str(i)] = features_train[['d1_lh_'+str(i),'d2_lh_'+str(i),'d3_lh_'+str(i),'d4_lh_'+str(i),'d5_lh_'+str(i)]].sum(axis=1) 
    features_train['gold_sum_diff_'+str(i)] = features_train['r_gold_sum_'+str(i)] - features_train['d_gold_sum_'+str(i)]
    features_train['gold_mean_diff_'+str(i)] = features_train['r_gold_mean_'+str(i)] - features_train['d_gold_mean_'+str(i)]
    features_train['xp_sum_diff_'+str(i)] = features_train['r_xp_sum_'+str(i)] - features_train['d_xp_sum_'+str(i)]
    features_train['xp_mean_diff_'+str(i)] = features_train['r_xp_mean_'+str(i)] - features_train['d_xp_mean_'+str(i)]
    features_train['level_sum_diff_'+str(i)] = features_train['r_level_sum_'+str(i)] - features_train['d_level_sum_'+str(i)]
    features_train['level_mean_diff_'+str(i)] = features_train['r_level_mean_'+str(i)] - features_train['d_level_mean_'+str(i)]
    features_train['lh_sum_diff_'+str(i)] = features_train['r_lh_sum_'+str(i)] - features_train['d_lh_sum_'+str(i)]
    features_train['lh_mean_diff_'+str(i)] = features_train['r_lh_mean_'+str(i)] - features_train['d_lh_mean_'+str(i)]

In [16]:
for i in [0,60,120,180,240,300]:
    features_test['r_level_mean_'+str(i)] = features_test[['r1_level_'+str(i),'r2_level_'+str(i),'r3_level_'+str(i),'r4_level_'+str(i),'r5_level_'+str(i)]].mean(axis=1)
    features_test['d_level_mean_'+str(i)] = features_test[['d1_level_'+str(i),'d2_level_'+str(i),'d3_level_'+str(i),'d4_level_'+str(i),'d5_level_'+str(i)]].mean(axis=1)
    features_test['r_xp_mean_'+str(i)] = features_test[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].mean(axis=1) 
    features_test['d_xp_mean_'+str(i)] = features_test[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].mean(axis=1)
    features_test['r_level_sum_'+str(i)] = features_test[['r1_level_'+str(i),'r2_level_'+str(i),'r3_level_'+str(i),'r4_level_'+str(i),'r5_level_'+str(i)]].sum(axis=1)
    features_test['d_level_sum_'+str(i)] = features_test[['d1_level_'+str(i),'d2_level_'+str(i),'d3_level_'+str(i),'d4_level_'+str(i),'d5_level_'+str(i)]].sum(axis=1)
    features_test['r_xp_sum_'+str(i)] = features_test[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].sum(axis=1) 
    features_test['d_xp_sum_'+str(i)] = features_test[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].sum(axis=1) 
    features_test['r_gold_mean_'+str(i)] = features_test[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].mean(axis=1)
    features_test['d_gold_mean_'+str(i)] = features_test[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].mean(axis=1) 
    features_test['r_gold_sum_'+str(i)] = features_test[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].sum(axis=1)
    features_test['d_gold_sum_'+str(i)] = features_test[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].sum(axis=1) 
    features_test['kills_diff_'+str(i)] = features_test[['r1_kills_'+str(i),'r2_kills_'+str(i),'r3_kills_'+str(i),'r4_kills_'+str(i),'r5_kills_'+str(i)]].sum(axis=1)-features_test[['d1_kills_'+str(i),'d2_kills_'+str(i),'d3_kills_'+str(i),'d4_kills_'+str(i),'d5_kills_'+str(i)]].sum(axis=1)
    features_test['r_lh_mean_'+str(i)] = features_test[['r1_lh_'+str(i),'r2_lh_'+str(i),'r3_lh_'+str(i),'r4_lh_'+str(i),'r5_lh_'+str(i)]].mean(axis=1) 
    features_test['d_lh_mean_'+str(i)] = features_test[['d1_lh_'+str(i),'d2_lh_'+str(i),'d3_lh_'+str(i),'d4_lh_'+str(i),'d5_lh_'+str(i)]].mean(axis=1) 
    features_test['r_lh_sum_'+str(i)] = features_test[['r1_lh_'+str(i),'r2_lh_'+str(i),'r3_lh_'+str(i),'r4_lh_'+str(i),'r5_lh_'+str(i)]].sum(axis=1) 
    features_test['d_lh_sum_'+str(i)] = features_test[['d1_lh_'+str(i),'d2_lh_'+str(i),'d3_lh_'+str(i),'d4_lh_'+str(i),'d5_lh_'+str(i)]].sum(axis=1) 
    features_test['gold_sum_diff_'+str(i)] = features_test['r_gold_sum_'+str(i)] - features_test['d_gold_sum_'+str(i)]
    features_test['gold_mean_diff_'+str(i)] = features_test['r_gold_mean_'+str(i)] - features_test['d_gold_mean_'+str(i)]
    features_test['xp_sum_diff_'+str(i)] = features_test['r_xp_sum_'+str(i)] - features_test['d_xp_sum_'+str(i)]
    features_test['xp_mean_diff_'+str(i)] = features_test['r_xp_mean_'+str(i)] - features_test['d_xp_mean_'+str(i)]
    features_test['level_sum_diff_'+str(i)] = features_test['r_level_sum_'+str(i)] - features_test['d_level_sum_'+str(i)]
    features_test['level_mean_diff_'+str(i)] = features_test['r_level_mean_'+str(i)] - features_test['d_level_mean_'+str(i)]
    features_test['lh_sum_diff_'+str(i)] = features_test['r_lh_sum_'+str(i)] - features_test['d_lh_sum_'+str(i)]
    features_test['lh_mean_diff_'+str(i)] = features_test['r_lh_mean_'+str(i)] - features_test['d_lh_mean_'+str(i)]

In [17]:
for i in [0,60,120,180,240,300]:
    features_test['r_level_median_'+str(i)] = features_test[['r1_level_'+str(i),'r2_level_'+str(i),'r3_level_'+str(i),'r4_level_'+str(i),'r5_level_'+str(i)]].median(axis=1)
    features_test['d_level_median_'+str(i)] = features_test[['d1_level_'+str(i),'d2_level_'+str(i),'d3_level_'+str(i),'d4_level_'+str(i),'d5_level_'+str(i)]].median(axis=1)
    features_test['r_xp_median_'+str(i)] = features_test[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].median(axis=1) 
    features_test['d_xp_median_'+str(i)] = features_test[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].median(axis=1) 
    features_test['r_xp_std_'+str(i)] = features_test[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].std(axis=1) 
    features_test['d_xp_std_'+str(i)] = features_test[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].std(axis=1) 
    features_test['r_lh_median_'+str(i)] = features_test[['r1_lh_'+str(i),'r2_lh_'+str(i),'r3_lh_'+str(i),'r4_lh_'+str(i),'r5_lh_'+str(i)]].median(axis=1)
    features_test['d_lh_median_'+str(i)] = features_test[['d1_lh_'+str(i),'d2_lh_'+str(i),'d3_lh_'+str(i),'d4_lh_'+str(i),'d5_lh_'+str(i)]].median(axis=1) 
    features_test['r_lh_std_'+str(i)] = features_test[['r1_lh_'+str(i),'r2_lh_'+str(i),'r3_lh_'+str(i),'r4_lh_'+str(i),'r5_lh_'+str(i)]].std(axis=1)
    features_test['d_lh_std_'+str(i)] = features_test[['d1_lh_'+str(i),'d2_lh_'+str(i),'d3_lh_'+str(i),'d4_lh_'+str(i),'d5_lh_'+str(i)]].std(axis=1) 
    features_test['r_gold_median_'+str(i)] = features_test[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].median(axis=1)
    features_test['d_gold_median_'+str(i)] = features_test[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].median(axis=1) 
    features_test['r_gold_std_'+str(i)] = features_test[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].std(axis=1)
    features_test['d_gold_std_'+str(i)] = features_test[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].std(axis=1) 

In [18]:
for i in [0,60,120,180,240,300]:
    features_train['r_level_median_'+str(i)] = features_train[['r1_level_'+str(i),'r2_level_'+str(i),'r3_level_'+str(i),'r4_level_'+str(i),'r5_level_'+str(i)]].median(axis=1)
    features_train['d_level_median_'+str(i)] = features_train[['d1_level_'+str(i),'d2_level_'+str(i),'d3_level_'+str(i),'d4_level_'+str(i),'d5_level_'+str(i)]].median(axis=1)
    features_train['r_xp_median_'+str(i)] = features_train[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].median(axis=1) 
    features_train['d_xp_median_'+str(i)] = features_train[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].median(axis=1) 
    features_train['r_xp_std_'+str(i)] = features_train[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].std(axis=1) 
    features_train['d_xp_std_'+str(i)] = features_train[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].std(axis=1)
    features_train['r_lh_median_'+str(i)] = features_train[['r1_lh_'+str(i),'r2_lh_'+str(i),'r3_lh_'+str(i),'r4_lh_'+str(i),'r5_lh_'+str(i)]].median(axis=1)
    features_train['d_lh_median_'+str(i)] = features_train[['d1_lh_'+str(i),'d2_lh_'+str(i),'d3_lh_'+str(i),'d4_lh_'+str(i),'d5_lh_'+str(i)]].median(axis=1) 
    features_train['r_lh_std_'+str(i)] = features_train[['r1_lh_'+str(i),'r2_lh_'+str(i),'r3_lh_'+str(i),'r4_lh_'+str(i),'r5_lh_'+str(i)]].std(axis=1)
    features_train['d_lh_std_'+str(i)] = features_train[['d1_lh_'+str(i),'d2_lh_'+str(i),'d3_lh_'+str(i),'d4_lh_'+str(i),'d5_lh_'+str(i)]].std(axis=1) 
    features_train['r_gold_median_'+str(i)] = features_train[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].median(axis=1)
    features_train['d_gold_median_'+str(i)] = features_train[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].median(axis=1) 
    features_train['r_gold_std_'+str(i)] = features_train[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].std(axis=1)
    features_train['d_gold_std_'+str(i)] = features_train[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].std(axis=1) 

In [19]:
for i in [60,120,180,240,300]:
    features_test['r_xp_mean_rate_'+str(i)] = features_test[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].mean(axis=1)/(i/60.0) 
    features_test['d_xp_mean_rate_'+str(i)] = features_test[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].mean(axis=1)/(i/60.0) 
    features_test['r_xp_sum_rate_'+str(i)] = features_test[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].sum(axis=1)/(i/60.0) 
    features_test['d_xp_sum_rate_'+str(i)] = features_test[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].sum(axis=1)/(i/60.0) 
    features_test['r_gold_mean_rate_'+str(i)] = features_test[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].mean(axis=1)/(i/60.0)
    features_test['d_gold_mean_rate_'+str(i)] = features_test[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].mean(axis=1)/(i/60.0) 
    features_test['r_gold_sum_rate_'+str(i)] = features_test[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].sum(axis=1)/(i/60.0)
    features_test['d_gold_sum_rate_'+str(i)] = features_test[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].sum(axis=1)/(i/60.0)
    features_test['r_level_mean_rate_'+str(i)] = features_test[['r1_level_'+str(i),'r2_level_'+str(i),'r3_level_'+str(i),'r4_level_'+str(i),'r5_level_'+str(i)]].mean(axis=1)/(i/60.0)
    features_test['d_level_mean_rate_'+str(i)] = features_test[['d1_level_'+str(i),'d2_level_'+str(i),'d3_level_'+str(i),'d4_level_'+str(i),'d5_level_'+str(i)]].mean(axis=1)/(i/60.0) 
    features_test['r_level_sum_rate_'+str(i)] = features_test[['r1_level_'+str(i),'r2_level_'+str(i),'r3_level_'+str(i),'r4_level_'+str(i),'r5_level_'+str(i)]].sum(axis=1)/(i/60.0)
    features_test['d_level_sum_rate_'+str(i)] = features_test[['d1_level_'+str(i),'d2_level_'+str(i),'d3_level_'+str(i),'d4_level_'+str(i),'d5_level_'+str(i)]].sum(axis=1)/(i/60.0)
    features_train['r_xp_mean_rate_'+str(i)] = features_train[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].mean(axis=1)/(i/60.0) 
    features_train['d_xp_mean_rate_'+str(i)] = features_train[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].mean(axis=1)/(i/60.0) 
    features_train['r_xp_sum_rate_'+str(i)] = features_train[['r1_xp_'+str(i),'r2_xp_'+str(i),'r3_xp_'+str(i),'r4_xp_'+str(i),'r5_xp_'+str(i)]].sum(axis=1)/(i/60.0) 
    features_train['d_xp_sum_rate_'+str(i)] = features_train[['d1_xp_'+str(i),'d2_xp_'+str(i),'d3_xp_'+str(i),'d4_xp_'+str(i),'d5_xp_'+str(i)]].sum(axis=1)/(i/60.0)
    features_train['r_level_mean_rate_'+str(i)] = features_train[['r1_level_'+str(i),'r2_level_'+str(i),'r3_level_'+str(i),'r4_level_'+str(i),'r5_level_'+str(i)]].mean(axis=1)/(i/60.0)
    features_train['d_level_mean_rate_'+str(i)] = features_train[['d1_level_'+str(i),'d2_level_'+str(i),'d3_level_'+str(i),'d4_level_'+str(i),'d5_level_'+str(i)]].mean(axis=1)/(i/60.0) 
    features_train['r_level_sum_rate_'+str(i)] = features_train[['r1_level_'+str(i),'r2_level_'+str(i),'r3_level_'+str(i),'r4_level_'+str(i),'r5_level_'+str(i)]].sum(axis=1)/(i/60.0)
    features_train['d_level_sum_rate_'+str(i)] = features_train[['d1_level_'+str(i),'d2_level_'+str(i),'d3_level_'+str(i),'d4_level_'+str(i),'d5_level_'+str(i)]].sum(axis=1)/(i/60.0)
    features_train['r_gold_mean_rate_'+str(i)] = features_train[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].mean(axis=1)/(i/60.0)
    features_train['d_gold_mean_rate_'+str(i)] = features_train[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].mean(axis=1)/(i/60.0) 
    features_train['r_gold_sum_rate_'+str(i)] = features_train[['r1_gold_'+str(i),'r2_gold_'+str(i),'r3_gold_'+str(i),'r4_gold_'+str(i),'r5_gold_'+str(i)]].sum(axis=1)/(i/60.0)
    features_train['d_gold_sum_rate_'+str(i)] = features_train[['d1_gold_'+str(i),'d2_gold_'+str(i),'d3_gold_'+str(i),'d4_gold_'+str(i),'d5_gold_'+str(i)]].sum(axis=1)/(i/60.0)

## далее идут вредные фичи, которые снижают качество до 0.75 

In [20]:
from sklearn import ensemble #импортируем ensemble
import sklearn.cross_validation #импортируем кросс-валидацию

# Логистическая регрессия
В это части рассматривается решение с помощью логистической регрессии.

In [21]:
from sklearn.preprocessing import StandardScaler #импортируем Scaler для нормализации признаков

In [22]:
scaler = StandardScaler() #инициализируем scaler
scaled_train = scaler.fit_transform(features_train) #нормализация train
scaled_test = scaler.fit_transform(features_test) #нормализация test

In [23]:
from sklearn.linear_model import LogisticRegression #имортируем логисчтиескую регрессию

In [24]:
features_train_copy = features_train.drop(['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero'],axis=1)
#убрали тип лобби и выбранных героев из данных

In [25]:
scaled_train_copy = scaler.fit_transform(features_train_copy) #scaling для набора данных без информации о героях

In [26]:
unique_heroes = []
for name in ['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']:
    for u in pd.unique(features_train[name]):
        if u not in unique_heroes:
            unique_heroes.append(u)
print 'len of unique heroes sequence:', len(unique_heroes)
print 'min and max hero id:',min(unique_heroes),'-', max(unique_heroes)

len of unique heroes sequence: 108
min and max hero id: 1 - 112


## Всего в игре 112 персонажей. Часть персонажей почему-то не была взята ни разу.
## Id персонажей заканчивается на значении 112.

In [27]:
#bag of words из примера
X_pick = np.zeros((features_train.shape[0], 112))

for i, match_id in enumerate(features_train.index):
    for p in xrange(5):
        X_pick[i, features_train.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_train.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [26]:
X_pair_pick = np.zeros((features_train.shape[0], 6105))
n_iter = 0

for i in xrange(112):
    for j in xrange(i+1,111):
        X_pair_pick[:, n_iter] = X_pick[:,i] + X_pick[:,j]
        n_iter += 1

In [27]:
X_pair_pick[X_pair_pick==1]=0
X_pair_pick[X_pair_pick==-1]=0
X_pair_pick[X_pair_pick==2]=1
X_pair_pick[X_pair_pick==-2]=-1

In [28]:
scaled_pick = scaler.fit_transform(X_pick) #scaling для pick'а
#scaled_pair_pick = scaler.fit_transform(X_pair_pick)

In [29]:
#scaled_train_bag = np.hstack((scaled_train_copy,scaled_pick,scaled_pair_pick)) #объединяем информацию
scaled_train_bag = np.hstack((scaled_train_copy,scaled_pick))

In [30]:
del(features_train_copy)
del(scaled_train_copy)

In [32]:
scaled_train_bag

array([[ -1.73252632e+00,  -2.54436416e+00,  -5.62011691e-01, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       [ -1.73249602e+00,  -2.54045236e+00,   1.77932242e+00, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       [ -1.73246573e+00,  -2.53923104e+00,  -5.62011691e-01, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       ..., 
       [  1.73353115e+00,   1.09874571e+00,  -5.62011691e-01, ...,
         -4.83865990e-03,   0.00000000e+00,   1.64609695e+00],
       [  1.73356145e+00,   1.09895204e+00,  -5.62011691e-01, ...,
         -4.83865990e-03,   0.00000000e+00,   8.46061344e-04],
       [  1.73359175e+00,   1.10264790e+00,  -5.62011691e-01, ...,
         -4.83865990e-03,   0.00000000e+00,  -1.64440483e+00]])

In [33]:
from sklearn import ensemble #импортируем ensemble
import sklearn.cross_validation #импортируем кросс-валидацию

In [34]:
#c_est1 = np.linspace(2.0,10.0,num=10)
#c_est2 = np.linspace(0.0001,0.001,num=10)
c_est3 = np.linspace(0.002,0.01,num=10)
c_est4 = np.linspace(0.02,0.1,num=10)
#c_est5 = np.linspace(0.2,1.0,num=10)
#c_est6 = np.linspace(11.0,100,num=10)
#c_est7 = np.linspace(101,1000,num=10)
#c_est8 = np.linspace(1001,10000,num=10)

In [35]:
c_est = []
#for i in c_est1:
#    c_est.append(i)
#for i in c_est2:
#    c_est.append(i)
for i in c_est3:
    c_est.append(i)    
for i in c_est4:
    c_est.append(i)
#for i in c_est5:
#    c_est.append(i)
#for i in c_est6:
#    c_est.append(i)
#for i in c_est7:
#    c_est.append(i)
#for i in c_est8:
#    c_est.append(i)
#print len(c_est)

In [36]:
import time
import datetime
#c = 0.00137777777778
bag_score = []#список результатов
c_est = np.array(c_est)#значения C для перебора
#c_est = [0.0000001,0.000001,0.00001,0.0001]
lr_models = []
for c in c_est:
    start_time = datetime.datetime.now()
    logr = LogisticRegression(penalty='l2',C=c,tol=0.000001,verbose=True)#логистическая регрессия
    lr_models.append(logr.fit(scaled_train_bag, features_target))#обучаем класификатор
    #bag_score.append(sklearn.cross_validation.cross_val_score(logr,scaled_train_bag, features_target,cv=kf, scoring='roc_auc').mean())
    #к списку результатов добавляем средний получившийся на кросс-валидации результат по roc_auc score
    #print bag_score
    print 'Time elapsed for c =', c,':', datetime.datetime.now() - start_time

[LibLinear]Time elapsed for c = 0.002 : 0:00:56.851265
[LibLinear]Time elapsed for c = 0.00288888888889 : 0:01:03.815284
[LibLinear]Time elapsed for c = 0.00377777777778 : 0:01:07.862754
[LibLinear]Time elapsed for c = 0.00466666666667 : 0:01:08.712091
[LibLinear]Time elapsed for c = 0.00555555555556 : 0:01:15.737719
[LibLinear]Time elapsed for c = 0.00644444444444 : 0:01:17.882037
[LibLinear]Time elapsed for c = 0.00733333333333 : 0:01:21.165129
[LibLinear]Time elapsed for c = 0.00822222222222 : 0:01:22.101299
[LibLinear]Time elapsed for c = 0.00911111111111 : 0:01:22.001495
[LibLinear]Time elapsed for c = 0.01 : 0:01:23.045296
[LibLinear]Time elapsed for c = 0.02 : 0:01:35.739685
[LibLinear]Time elapsed for c = 0.0288888888889 : 0:01:42.887511
[LibLinear]Time elapsed for c = 0.0377777777778 : 0:01:50.495936
[LibLinear]Time elapsed for c = 0.0466666666667 : 0:01:48.557039
[LibLinear]Time elapsed for c = 0.0555555555556 : 0:01:52.067226
[LibLinear]Time elapsed for c = 0.0644444444444 :

In [36]:
#проделаем над тестовым набором те же преобразования, что и над обучающим

In [37]:
features_train.head()

,match_id,start_time,lobby_type,r1_hero,r1_level_0,r1_xp_0,r1_gold_0,r1_lh_0,r1_kills_0,r1_deaths_0,...,r_xp_sum_rate_300,d_xp_sum_rate_300,r_level_mean_rate_300,d_level_mean_rate_300,r_level_sum_rate_300,d_level_sum_rate_300,r_gold_mean_rate_300,d_gold_mean_rate_300,r_gold_sum_rate_300,d_gold_sum_rate_300
0,0,1430198770,7,11,0,0,0,0,0,0,...,1267.6,1179.0,0.76,0.72,3.8,3.6,200.88,200.84,1004.4,1004.2
1,1,1430220345,0,42,1,0,0,0,0,0,...,1074.8,1046.0,0.68,0.68,3.4,3.4,187.92,220.20,939.6,1101.0
2,2,1430227081,7,33,0,0,0,0,0,0,...,1537.0,1350.6,0.72,0.72,3.6,3.6,267.08,231.44,1335.4,1157.2
3,3,1430263531,1,29,0,0,0,0,0,0,...,1131.6,1095.0,0.64,0.64,3.2,3.2,172.32,189.16,861.6,945.8
4,4,1430282290,7,13,1,0,0,0,0,0,...,1070.2,1172.4,0.64,0.72,3.2,3.6,195.52,202.72,977.6,1013.6


In [38]:
features_test_copy = features_test.drop(['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero'],axis=1)

In [39]:
scaled_test_copy = scaler.fit_transform(features_test_copy) #scaling для набора данных без информации о героях

In [40]:
#bag of words из примера
X_test_pick = np.zeros((features_test.shape[0], 112))

for i, match_id in enumerate(features_test.index):
    for p in xrange(5):
        X_test_pick[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_test_pick[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [42]:
X_test_pair_pick = np.zeros((features_test.shape[0], 6105))
n_iter = 0

for i in xrange(112):
    for j in xrange(i+1,111):
        X_test_pair_pick[:, n_iter] = X_test_pick[:,i] + X_test_pick[:,j]
        n_iter += 1

In [43]:
X_test_pair_pick[X_test_pair_pick==1]=0
X_test_pair_pick[X_test_pair_pick==-1]=0
X_test_pair_pick[X_test_pair_pick==2]=1
X_test_pair_pick[X_test_pair_pick==-2]=-1

In [41]:
scaled_test_pick = scaler.fit_transform(X_test_pick) #scaling для pick'а
#scaled_test_pair_pick = scaler.fit_transform(X_test_pair_pick)

In [42]:
#scaled_test_bag = np.hstack((scaled_test_copy,scaled_test_pick, scaled_test_pair_pick)) #объединяем информацию
scaled_test_bag = np.hstack((scaled_test_copy,scaled_test_pick))

In [43]:
for i in range(len(lr_models)):
    if i==0:
        result = lr_models[i].predict_proba(scaled_test_bag)[:,1]
    else:
        probabilities = lr_models[i].predict_proba(scaled_test_bag)[:,1]
        result += probabilities

In [44]:
result = result.transpose()      

In [45]:
result

array([ 16.41360962,  15.6127974 ,   3.89469046, ...,   4.80060679,
        14.98000624,   9.29979873])

In [46]:
result = result/len(lr_models)

In [47]:
result

array([ 0.82068048,  0.78063987,  0.19473452, ...,  0.24003034,
        0.74900031,  0.46498994])

In [48]:
#result=pd.DataFrame(result)

In [49]:
test = pd.read_csv('features_test.csv')#считывание test_features

In [50]:
test_id = test['match_id']

In [51]:
record = np.vstack((test_id,result)).transpose()

In [52]:
record

array([[  6.00000000e+00,   8.20680481e-01],
       [  7.00000000e+00,   7.80639870e-01],
       [  1.00000000e+01,   1.94734523e-01],
       ..., 
       [  1.14378000e+05,   2.40030339e-01],
       [  1.14393000e+05,   7.49000312e-01],
       [  1.14398000e+05,   4.64989936e-01]])

In [53]:
record=pd.DataFrame(record)

In [54]:
record[0]=record[0].astype(int)

In [55]:
record.to_csv('result.csv', header=["match_id","radiant_win"],index=False)